In [1]:
#####
# 1.最新四半期決算の営業利益率は低い
# 2.最新四半期決算の売上高成長率が10%以上
# 3.最新四半期決算の対売上高営業利益成長率が20%以上
# 4.次期決算を、以下の方法で予測した場合において、営業利益率が前年同期比2倍以上になる
# --> 最新四半期決算の売上高成長率と対売上高営業利益成長率を、次期前年四半期決算に適用して次期決算を予想する

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig, IndexPricelistPl
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

RevPl = PricelistPl("reviced_pricelist.parquet")
RawPL = PricelistPl("raw_pricelist.parquet")
MPL = MeigaralistPl()
SPL = ShikihoPl()

In [3]:
KPL = KessanPl()
KPL.with_columns_lastyear_settlement()
df1 = KPL.df.filter(pl.col("settlement_type")=="四")
df1 = df1.with_columns([
    pl.col("settlement_date").alias("key")
])

In [4]:
# para
valuation_date = date(2024, 7, 1)

KPL = KessanPl()
df2 = KPL.get_settlement_forcast_by_diff_growth_rate(valuation_date)
df2 = df2.filter(pl.col("settlement_type")=="四")
df2 = df2.with_columns([
    pl.col("nxt_settlement_date").alias("key")
])

In [5]:
KPL = KessanPl()
df3 = KPL.get_current_settlements(valuation_date)
df3 = df3.filter(pl.col("settlement_type")=="四")
df3 = df3.with_columns([
    pl.col("settlement_date").alias("key")
])

In [6]:
KPL = KessanPl()
df = KPL.get_settlement_updown_rate("四")
df = df.with_columns([
    pl.col("settlement_date").alias("key")
])
df4 = df.select(["code", "key", "updown_rate", "nh_updown_rate"])
# df4.filter(pl.col("code")==9022)

In [7]:
# df1, df2, df3, df4をjoin&select
df = df2.join(df1, on=["code", "key"], how="left")
df21 = df

df = df21.with_columns([
    pl.col("settlement_date").alias("lst_settlement_date"),
    pl.col("announcement_date").alias("lst_announcement_date"),
    pl.col("settlement_type").alias("lst_settlement_type"),
    pl.col("sales").alias("lst_sales"),
    pl.col("operating_income").alias("lst_operating_income"),
    pl.col("ordinary_profit").alias("lst_ordinary_profit"),
    pl.col("final_profit").alias("lst_final_profit"),
    pl.col("quater").alias("lst_quater")
])
df = df.select([
    "code",	
    "lst_settlement_date",
    "lst_announcement_date",
    "lst_settlement_type",
    'lst_sales',
    'lst_operating_income',
    'lst_ordinary_profit',
    'lst_final_profit',
    "quater",
    "ly_settlement_date",
    "ly_sales",
    "ly_operating_income",
    "ly_ordinary_profit",
    "ly_final_profit",
    "nxt_settlement_date",
    "fcst_dgr_sales",
    "fcst_dgr_operating_income",
    "fcst_dgr_ordinary_profit",	
    "fcst_dgr_final_profit",
    "key"
])
df = df.join(df3, on=["code", "key"], how="left")
df = df.drop_nulls()

df123 = df

In [8]:
# ly_fcstの売上高、営業利益成長率を
# 列名ly_fcst_sales_gr, ly_fcst_operating_income_grで追加
df = df123.with_columns([
    (pl.lit(100) * pl.col("lst_operating_income") / pl.col("lst_sales")).round(2).alias("lst_operating_income_pr"),
    (pl.lit(100) * (pl.col("fcst_dgr_sales") - pl.col("ly_sales")) / pl.col("ly_sales")).round(2).alias("ly_fcst_sales_gr"),
    (pl.lit(100) * (pl.col("fcst_dgr_operating_income") - pl.col("ly_operating_income")) / pl.col("ly_operating_income")).round(2).alias("ly_fcst_operating_income_gr"),
])

df = df.join(df4, on=["code", "key"], how="left")
df = df.drop_nulls()

scols = [
    "code",
    "lst_settlement_date",
    #lst_announcement_date,
    #lst_settlement_type,
    "lst_sales",
    "lst_operating_income",
    "lst_ordinary_profit",
    "lst_final_profit",
    "quater",
    #ly_settlement_date,
    "ly_sales",
    "ly_operating_income",
    #ly_ordinary_profit,
    #ly_final_profit,
    #nxt_settlement_date,
    "fcst_dgr_sales",
    "fcst_dgr_operating_income",
    "fcst_dgr_ordinary_profit",
    "fcst_dgr_final_profit",
    #key,
    #settlement_date,
    #settlement_type,
    #announcement_date,
    "sales",
    "operating_income",
    "ordinary_profit",
    "final_profit",
    #reviced_eps,
    #dividend,
    #quater_right,
    "lst_operating_income_pr",
    "ly_fcst_sales_gr",
    "ly_fcst_operating_income_gr",
    "updown_rate",
    "nh_updown_rate"
]
df = df.select(scols)
codes = df["code"].unique()
df = df.sort(by=["ly_fcst_operating_income_gr"], descending=[True])
original_df = df
original_df

code,lst_settlement_date,lst_sales,lst_operating_income,lst_ordinary_profit,lst_final_profit,quater,ly_sales,ly_operating_income,fcst_dgr_sales,fcst_dgr_operating_income,fcst_dgr_ordinary_profit,fcst_dgr_final_profit,sales,operating_income,ordinary_profit,final_profit,lst_operating_income_pr,ly_fcst_sales_gr,ly_fcst_operating_income_gr,updown_rate,nh_updown_rate
i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64
4406,2024-03-31,8783,456,476,303,4,7952,1,9986,1227,1205,896,8194,203,281,236,5.19,25.58,122600.0,4.8,-7.12
9438,2024-03-31,6892,617,726,928,2,6500,1,7271,1227,1206,1633,7019,700,845,685,8.95,11.86,122600.0,3.5,-11.05
4418,2024-03-31,5531,41,28,8,3,452,1,55022,833,721,651,5292,4,-12,-213,0.74,12073.01,83200.0,-13.7,-4.39
5280,2024-03-31,15275,2712,2797,1740,4,4731,16,44496,8644,8583,5188,3613,-286,-290,-209,17.75,840.52,53925.0,5.4,0.38
6145,2024-03-31,11465,2173,2165,1395,4,4195,9,16383,3727,3852,2320,4319,-611,-484,-345,18.95,290.54,41311.11,-27.0,-7.12
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
8944,2024-03-31,2477,-365,-454,-321,2,1105,7,5387,-1244,-1289,-2512,3158,-288,-386,-291,-14.74,387.51,-17871.43,-17.1,-8.52
1961,2024-03-31,72309,5588,5770,4090,4,40727,-18,80263,7083,7472,4527,48527,1540,2087,1345,7.73,97.08,-39450.0,2.6,-7.12
7494,2024-03-31,19278,1143,1145,653,2,16144,-2,19104,1514,1457,811,14560,-599,-696,-829,5.93,18.33,-75800.0,-28.4,-5.86


In [9]:
######
###### filter
######

In [10]:
min_ly_operating_income = -999999999
max_ly_operating_income = 0

min_lst_operating_income_pr = -99999999
max_lst_operating_income_pr = 0

min_ly_fcst_sales_gr = 10
max_ly_fcst_sales_gr = 999999999

min_ly_fcst_operating_income_gr = -999999999	
max_ly_fcst_operating_income_gr	= 0

min_fcst_dgr_operating_income = 0
max_fcst_dgr_operating_income = 999999999


In [11]:
df = original_df

df = df\
    .filter(pl.col("ly_operating_income") >= min_ly_operating_income)\
    .filter(pl.col("ly_operating_income")  < max_ly_operating_income)\
\
    .filter(pl.col("lst_operating_income_pr") >= min_lst_operating_income_pr)\
    .filter(pl.col("lst_operating_income_pr") < max_lst_operating_income_pr)\
\
    .filter(pl.col("ly_fcst_sales_gr") >= min_ly_fcst_sales_gr)\
    .filter(pl.col("ly_fcst_sales_gr") < max_ly_fcst_sales_gr)\
\
    .filter(pl.col("ly_fcst_operating_income_gr") >= min_ly_fcst_operating_income_gr)\
    .filter(pl.col("ly_fcst_operating_income_gr") < max_ly_fcst_operating_income_gr)\
\
    .filter(pl.col("fcst_dgr_operating_income") >= min_fcst_dgr_operating_income)\
    .filter(pl.col("fcst_dgr_operating_income") <  max_fcst_dgr_operating_income)

filtered_df = df
pl.Config(tbl_rows=df.shape[0])
pl.Config.set_tbl_cols(df.shape[1])
df

code,lst_settlement_date,lst_sales,lst_operating_income,lst_ordinary_profit,lst_final_profit,quater,ly_sales,ly_operating_income,fcst_dgr_sales,fcst_dgr_operating_income,fcst_dgr_ordinary_profit,fcst_dgr_final_profit,sales,operating_income,ordinary_profit,final_profit,lst_operating_income_pr,ly_fcst_sales_gr,ly_fcst_operating_income_gr,updown_rate,nh_updown_rate
i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64
2397,2024-03-31,219,-23,-9,-11,4,67,-95,300,1,38,39,206,-19,-19,-20,-10.5,347.76,-101.05,-23.3,-7.69
6085,2024-03-31,175,-38,-55,-122,4,127,-65,199,1,-34,-150,102,-75,-76,-83,-21.71,56.69,-101.54,8.2,-7.31
6696,2024-04-30,82,-8,-8,-9,1,48,-39,91,4,6,3,80,-10,-10,-11,-9.76,89.58,-110.26,-39.2,-6.87
4375,2024-03-31,3446,-99,-117,-126,1,2900,-257,4704,27,-13,-33,3399,-203,-212,-1045,-2.87,62.21,-110.51,4.5,-8.16
2934,2024-02-29,4870,-110,-107,-128,3,4124,-1126,6555,193,206,59,5337,-12,-91,-1449,-2.26,58.95,-117.14,-50.1,2.03
1815,2024-03-31,50928,-3,503,227,4,39660,-1398,56683,285,1222,-538,44057,269,566,2085,-0.01,42.92,-120.39,-9.2,-7.9
4755,2024-03-31,513624,-33272,-28469,-42394,1,497165,-48893,554714,13571,27044,1449,537284,-18331,-14837,-33568,-6.48,11.58,-127.76,-1.6,-7.9
6046,2024-03-31,243,-26,-26,-27,2,221,-67,287,20,18,16,251,-14,-13,-13,-10.7,29.86,-129.85,-13.8,-7.12
4056,2024-03-31,780,-100,-114,-109,1,772,-221,1009,67,42,-3,881,-5,-6,-36,-12.82,30.7,-130.32,-21.2,-7.9


In [12]:
df = original_df

print("##### 全体")
print(f'総銘柄数：{df.shape[0]}')
a = df.filter(pl.col("updown_rate")>0).shape[0]
b = df.filter(pl.col("updown_rate")<=0).shape[0]
uprate = round(100*a/(a+b), 1)
print(f'上昇銘柄数:{a}／下落銘柄数:{b}／上昇率:({uprate}%)')
print(f'全銘柄平均騰落率:{round(df["updown_rate"].mean(), 2)}%')
print(f'日経225平均騰落率:{round(df["nh_updown_rate"].mean(), 2)}%')
print()

df = filtered_df
print("##### 抽出銘柄")
print(f'総銘柄数：{df.shape[0]}')
a = df.filter(pl.col("updown_rate")>0).shape[0]
b = df.filter(pl.col("updown_rate")<=0).shape[0]
uprate = round(100*a/(a+b), 1)
print(f'上昇銘柄数:{a}／下落銘柄数:{b}／上昇率:({uprate}%)')
print(f'抽出銘柄平均騰落率:{round(df["updown_rate"].mean(), 2)}%')
print(f'日経225平均騰落率:{round(df["nh_updown_rate"].mean(), 2)}%')

##### 全体
総銘柄数：3610
上昇銘柄数:1083／下落銘柄数:2527／上昇率:(30.0%)
全銘柄平均騰落率:-4.6%
日経225平均騰落率:-4.85%

##### 抽出銘柄
総銘柄数：29
上昇銘柄数:10／下落銘柄数:19／上昇率:(34.5%)
抽出銘柄平均騰落率:-0.06%
日経225平均騰落率:-5.67%


In [13]:
#####
##### 個別銘柄の分析
#####

In [14]:
# 四季報
code = 4100
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

4100(戸田工業)の四季報データ履歴

発行日: 2024年06月14日
【上向く】
  韓国持分の連結化で売上47億円、利益1億円上乗せ。複写機・プリンタ向け顔料戻り鈍いが、磁石・誘電体材料が下期から復調。前期の保有資産減損(48億円)効く。営業益好転。為替差益見込まず。
【次世代】
  磁石材料手がける中国子会社が冷却用ポンプでEVバッテリーの熱管理システムに食い込み。定置用蓄電池を念頭にナトリウムイオン電池を大学と基礎研究。

発行日: 2024年03月16日
【反発】
  連結除外で売上40億円、利益3億円剥落。顔料、電材とも数量回復遅い。円安効果で減益幅縮小。25年3月期は韓国持分の連結化で売上47億円、利益1億円上乗せ。複写機向け等顔料は軟調も、電子素材が復調。利益上向く。段階取得差益ない。
【注力】
  増設中の2次電池正極材は出荷先の認証取得後、24年中に商業生産。車載MLCC用に独自の分散体サンプル出荷。

発行日: 2023年12月15日
【減額】
  中国子会社の連結除外通期化で売上高40億円、営業益3億円剥落。複写機向け顔料や電池材料、誘電体材料が想定届かず、減益幅拡大。25年3月期は自動車向け磁石材料が堅調。韓国の子会社化も寄与。
【子会社化】
  韓国の持分法適用会社を23年11月子会社化、自動車向け軟磁性材料の開発を一気通貫し市場拡大に対応。持分の正極材料の需要増を受け、山口県の工場増強。

発行日: 2023年09月15日
【下振れ】
  中国子会社の連結除外通期化で売上高40億円、営業益3億円剥落。自動車向け磁石材料は顧客生産台数が下期回復し横ばい。複写機向け顔料は想定以下。原材料価格の低下強く見込む会社営業益過大。
【延期】
  プライム上場維持基準の適合計画期間を24年度末へ1年延期、既存事業の回復等で流通株式時価総額適合を目指す。8月より北海道で水素供給網の実証実験。


In [15]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [16]:
# 日足
delta = relativedelta(days=100)

KPL = KessanPl()
df = KPL.df.filter(pl.col("announcement_date")<valuation_date).filter(pl.col("code")==code)
ann_date = df["announcement_date"].to_list()[-1]
print(f'決算発表日: {ann_date}')
start_date = ann_date - delta
end_date = ann_date + delta

PFIG = PricelistFig(code, RevPl.df, MPL.df, start_date, end_date)
# 決算発表日にvlineを引く
PFIG.add_vline_announcement_date()
PFIG.add_vline(valuation_date)
PFIG.fig.show()


決算発表日: 2024-05-15
